In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from datetime import date, timedelta
import pandas as pd
import os
# Initialize WebDriver
driver_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome()

# Define function to extract event details
def extract_event_details():
#     if driver.find_element(By.CLASS_NAME,'event-details__main-inner'):
#         button = driver.find_element(By.CSS_SELECTOR,"button[data-testid='view-event-details-button']")
#         button = button_div.find_element(By.TAG_NAME,'button')
#         button.click()
#         print('button clicked')
#         WebDriverWait(driver, 10)
#         d = driver.current_url
#         driver.get(d)
#         print("reloaded")
#         WebDriverWait(driver, 10)
        
   
    events_class = driver.find_element(By.CLASS_NAME, 'SearchResultPanelContentEventCardList-module__eventList___1YEh_')
    event_lists = []
    end_date = None
    events = []
    #print(events.text)
    #for event in events:
    event_hrefs = events_class.find_elements(By.TAG_NAME,'a')
    for x in range(0,len(event_hrefs),2):
        event_lists.append(event_hrefs[x].get_property('href'))
        #print(event_lists)
    for url in event_lists:
        driver.get(url)
        try:
            button = driver.find_element(By.CSS_SELECTOR,"button[data-testid='view-event-details-button']")
            #button = button_div.find_element(By.TAG_NAME,'button')
            button.click()
            print('button clicked')
            WebDriverWait(driver, 10)
            d = driver.current_url
            driver.get(d)
            print("reloaded")
            WebDriverWait(driver, 10)
        except:
            pass     
        title = driver.find_element(By.TAG_NAME,'h1').text
        #print(title.text)
        description = driver.find_element(By.CLASS_NAME,'summary').text
        #print(description)
        try:
            d = driver.find_element(By.CLASS_NAME,'date-info__full-datetime').text.split(',')
            if len(d) == 2:
                date_and_time_info = d[1]
                date,time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]


                s = time_info.split('-')
                if len(s)  == 2:
                    start_time,end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    #print(start_time)
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                    #print(end_time)

                else:
                    start_time = time_info
                    end_time = None

                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone

            elif len(d) > 2:
                start = d[1]
                start = start.split('-')[0]
                start = start.split()
                date = ' '.join(start[:3])
                start_time = start[-1]

                end = d[2]
                end = end.split()
                end_date = ' '.join(end[:3])
                end_time = end[-2]

                time_zone = end[-1]

                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone

            else:
                date_and_time_info = d[0]
                date,time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]


                s = time_info.split('-')
                if len(s)  == 2:
                    start_time,end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    #print(start_time)
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                    #print(end_time)

                else:
                    start_time = time_info
                    end_time = None

                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone

        except:
            date = driver.find_element(By.CLASS_NAME,'start-date').text.split(',')[1]
                
        if date.isalpha():
            if data.lower() == 'today':
                date = date.today()
            elif data.lower() == 'yesterday':
                date = date.today() - timedelta(days=1)
            elif data.lower() == 'monday':
                date = date.today() + timedelta(days=1)
            elif data.lower() == 'tuesday':
                date = date.today() + timedelta(days=2)
            elif data.lower() == 'wednesday':
                date = date.today() + timedelta(days=3)
            elif data.lower() == 'thursday':
                date = date.today() + timedelta(days=4)
            elif data.lower() == 'friday':
                date = date.today() + timedelta(days=5)
            elif data.lower() == 'saturday':
                date = date.today() + timedelta(days=6)
            elif data.lower() == 'sunday':
                date = date.today() + timedelta(days=7)
                

        
        #print(date, start_time,end_time)
        
        #loc = driver.find_element(By.CLASS_NAME,'location-info__address-text').text
        #print(loc)
        
        loc = driver.find_element(By.CLASS_NAME,'location-info__address').text.split('\n')
        #print(loc)
        venue = loc[0]
        address = loc[1]
        #print(venue, address)
        try:
            price = driver.find_element(By.CLASS_NAME,'conversion-bar__panel-info').text
        #print(price)
        except:
            price = None
        
        if not end_date:
            end_date = date
            
        events.append({
            'Event Title': title,
            'Event Description' :description,
            'Event Start Date': date,
            'Event End Date': end_date,
            'Event Start Time': start_time,
            'Event End Time': end_time,
            'Event Venue': venue,
            'Venue Address': address,
            'link': url
        })
        
        title = None
        description = None
        start_date = None
        end_date = None
        start_time = None
        end_time = None
        

    return events

all_events = []
link = r"https://www.eventbrite.com/d/tx--dallas/free--business--events/?page="

# Visit Eventbrite website
for i in range(4,61):
    path = link + str(i+1)
    driver.get(path)

    time.sleep(5)  # Adjust sleep time as necessary to ensure page loads

   

# Loop through pagination
# while True:
    all_events.extend(extract_event_details())
    print(f"Page {i+1} completed")
    
#     try:
#         next_button = driver.find_element(By.CLASS_NAME, 'button[aria-label="Next"]')
#         if next_button.is_enabled():
#             next_button.click()
#             time.sleep(5)  # Adjust sleep time as necessary to ensure page loads
#         else:
#             break
#     except Exception as e:
#         print(f"No more pages: {e}")
#         break

# # Save data to CSV
# keys = all_events[0].keys()
# with open('eventbrite_events.csv', 'w', newline='', encoding='utf-8') as output_file:
#     dict_writer = csv.DictWriter(output_file, fieldnames=keys)
#     dict_writer.writeheader()
#     dict_writer.writerows(all_events)

df = pd.DataFrame(all_events)
print(df)
df.to_csv('Dallas_Events.csv')
# Close the driver
driver.quit()

print("Scraping complete. Data saved to eventbrite_events.csv")


button clicked
Page 5 completed
button clicked
Page 6 completed


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".start-date"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF662E01F52+60322]
	(No symbol) [0x00007FF662D7CEC9]
	(No symbol) [0x00007FF662C37EBA]
	(No symbol) [0x00007FF662C87676]
	(No symbol) [0x00007FF662C8773C]
	(No symbol) [0x00007FF662CCE967]
	(No symbol) [0x00007FF662CAC25F]
	(No symbol) [0x00007FF662CCBC80]
	(No symbol) [0x00007FF662CABFC3]
	(No symbol) [0x00007FF662C79617]
	(No symbol) [0x00007FF662C7A211]
	GetHandleVerifier [0x00007FF6631194AD+3301629]
	GetHandleVerifier [0x00007FF6631636D3+3605283]
	GetHandleVerifier [0x00007FF663159450+3563680]
	GetHandleVerifier [0x00007FF662EB4326+790390]
	(No symbol) [0x00007FF662D8750F]
	(No symbol) [0x00007FF662D83404]
	(No symbol) [0x00007FF662D835

In [17]:
from datetime import date

today = date.today()
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_name = weekdays[today.weekday()]
print(day_name)  # Output: Wednesday


Wednesday


In [22]:
from datetime import date, timedelta
today - timedelta(days=5)

datetime.date(2024, 5, 31)

In [25]:
date_string = "Sunday, June 16·7:00 PM"

# Find the exact character
for char in date_string:
    if char not in "Sunday, June 167:00 PM":
        print(f"Character: {char} | Unicode: {ord(char)}")

Character: · | Unicode: 183


In [48]:
df.to_csv('Dallas_Events.csv')

In [49]:
df = pd.read_csv('Dallas_Events.csv')

In [50]:
df.head()

,Unnamed: 0,Event Title,Event Description,Event Date,Event Start Time,Event End Time,Event Venue,Venue Address,link
0,0,21st Annual Father’s Day Golf Outing Four Man ...,"Hey everyone, join us for a fun-filled day of ...",June 16,7am CDT,1pm CDT,Cedar Crest Golf Course,"1800 Southerland Avenue Dallas, TX 75203 Unite...",https://www.eventbrite.com/e/21st-annual-fathe...
1,1,Bella Modeling School Summer 2024 Fashion Show,Bella Modeling School Fashion Show will be fea...,June 6,6:30 CDT,9:30pm CDT,Lofty Spaces,"816 Montgomery Street Dallas, TX 75215 United ...",https://www.eventbrite.com/e/bella-modeling-sc...
2,2,SB Fashion Week Dallas Runway Show & Vendor Ma...,Join us for fashion's top runway show and mark...,June 15,7 CDT,9:30pm CDT,1515 S Ewing Ave,"1515 South Ewing Avenue Dallas, TX 75216 Unite...",https://www.eventbrite.com/e/sb-fashion-week-d...
3,3,Juneteenth Soul Food Festival,Celebrating the Juneteenth Festival with Great...,June 16,12 CDT,6pm CDT,Lofty Spaces,"816 Montgomery Street Dallas, TX 75215 United ...",https://www.eventbrite.com/e/juneteenth-soul-f...
4,4,Dallas Juneteenth Festival,Commenerating the Juneteenth Holiday with live...,June 16,12 CDT,6pm CDT,Lofty Spaces,"816 Montgomery Street Dallas, TX 75215 United ...",https://www.eventbrite.com/e/dallas-juneteenth...


In [51]:
len(df)

20

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from datetime import date, timedelta
import pandas as pd
import os
# Initialize WebDriver
driver_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome()

# Define function to extract event details
def extract_event_details():
    events_class = driver.find_element(By.CLASS_NAME, 'SearchResultPanelContentEventCardList-module__eventList___1YEh_')
    event_lists = []
    end_date = None
    events = []
    #print(events.text)
    #for event in events:
    event_li = events_class.find_elements(By.TAG_NAME,'li')
    for i in event_li:
        title = i.find_element(By.TAG_NAME,'h2').text
        other_data = i.find_elements(By.TAG_NAME,'p')
        date_and_time_info = other_data[0],text
        a = i.find_element(By.TAG_NAME,'a')
        href = a.get_property('href')
        driver2 =  webdriver.Chrome()
        driver2.get(href)
        try:
            d = date_and_time_info.split(',')
            if len(d) == 2:
                date_and_time_info = d[1]
                date,time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]


                s = time_info.split('-')
                if len(s)  == 2:
                    start_time,end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    #print(start_time)
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                    #print(end_time)

                else:
                    start_time = time_info
                    end_time = None

                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone

            elif len(d) > 2:
                start = d[1]
                start = start.split('-')[0]
                start = start.split()
                date = ' '.join(start[:3])
                start_time = start[-1]

                end = d[2]
                end = end.split()
                end_date = ' '.join(end[:3])
                end_time = end[-2]

                time_zone = end[-1]

                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone

            else:
                date_and_time_info = d[0]
                date,time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]


                s = time_info.split('-')
                if len(s)  == 2:
                    start_time,end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    #print(start_time)
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                    #print(end_time)

                else:
                    start_time = time_info
                    end_time = None

                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone

        except:
            date = date_and_time_info.split(',')[1]
            
            
        
                
        if date.isalpha():
            if data.lower() == 'today':
                date = date.today()
            elif data.lower() == 'yesterday':
                date = date.today() - timedelta(days=1)
            elif data.lower() == 'monday':
                date = date.today() + timedelta(days=1)
            elif data.lower() == 'tuesday':
                date = date.today() + timedelta(days=2)
            elif data.lower() == 'wednesday':
                date = date.today() + timedelta(days=3)
            elif data.lower() == 'thursday':
                date = date.today() + timedelta(days=4)
            elif data.lower() == 'friday':
                date = date.today() + timedelta(days=5)
            elif data.lower() == 'saturday':
                date = date.today() + timedelta(days=6)
            elif data.lower() == 'sunday':
                date = date.today() + timedelta(days=7)
                

        

        venue = other_data[1].text
        #print(venue, address)
        try:
            price = other_data.split()[1]

        #print(price)
        except:
            price = None
            
        driver.get()
        
        if not end_date:
            end_date = date
            
        events.append({
            'Event Title': title,
            'Event Description' :description,
            'Event Start Date': date,
            'Event End Date': end_date,
            'Event Start Time': start_time,
            'Event End Time': end_time,
            'Event Venue': venue,
            'Venue Address': address,
            'link': url
        })
        
        title = None
        description = None
        start_date = None
        end_date = None
        start_time = None
        end_time = None
        

    return events

all_events = []
link = r"https://www.eventbrite.com/d/tx--dallas/free--business--events/?page="

# Visit Eventbrite website
for i in range(4,61):
    path = link + str(i+1)
    driver.get(path)

    time.sleep(5)  # Adjust sleep time as necessary to ensure page loads

   

# Loop through pagination
# while True:
    all_events.extend(extract_event_details())
    print(f"Page {i+1} completed")
    
#     try:
#         next_button = driver.find_element(By.CLASS_NAME, 'button[aria-label="Next"]')
#         if next_button.is_enabled():
#             next_button.click()
#             time.sleep(5)  # Adjust sleep time as necessary to ensure page loads
#         else:
#             break
#     except Exception as e:
#         print(f"No more pages: {e}")
#         break

# # Save data to CSV
# keys = all_events[0].keys()
# with open('eventbrite_events.csv', 'w', newline='', encoding='utf-8') as output_file:
#     dict_writer = csv.DictWriter(output_file, fieldnames=keys)
#     dict_writer.writeheader()
#     dict_writer.writerows(all_events)

df = pd.DataFrame(all_events)
print(df)
df.to_csv('Dallas_Events.csv')
# Close the driver
driver.quit()

print("Scraping complete. Data saved to eventbrite_events.csv")


In [13]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import date, timedelta
import pandas as pd
import time

# Function to extract event details
def extract_event_details(soup, url):
    events = []
    event_lists = []
    
    # Find the event list
    events_class = soup.find(class_='SearchResultPanelContentEventCardList-module_eventList_1YEh')
    if not events_class:
        print(f"No events found on {url}")
        return events
    
    event_hrefs = events_class.find_all('a', href=True)
    for x in range(0, len(event_hrefs), 2):
        event_lists.append(event_hrefs[x]['href'])
    
    for event_url in event_lists:
        event_response = requests.get(event_url)
        event_soup = BeautifulSoup(event_response.content, 'html.parser')
        
        # Extract event details
        try:
            title = event_soup.find('h1').text
        except AttributeError:
            title = None
        
        try:
            description = event_soup.find(class_='summary').text
        except AttributeError:
            description = None
        
        try:
            d = event_soup.find(class_='date-info__full-datetime').text.split(',')
            if len(d) == 2:
                date_and_time_info = d[1]
                date, time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]
                s = time_info.split('-')
                if len(s) == 2:
                    start_time, end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                else:
                    start_time = time_info
                    end_time = None
                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone
            elif len(d) > 2:
                start = d[1].split('-')[0].split()
                date = ' '.join(start[:3])
                start_time = start[-1]
                end = d[2].split()
                end_date = ' '.join(end[:3])
                end_time = end[-2]
                time_zone = end[-1]
                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone
            else:
                date_and_time_info = d[0]
                date, time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]
                s = time_info.split('-')
                if len(s) == 2:
                    start_time, end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                else:
                    start_time = time_info
                    end_time = None
                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone
        except AttributeError:
            date = event_soup.find(class_='start-date').text.split(',')[1]
            end_date = None
        
        if date.isalpha():
            if date.lower() == 'today':
                date = date.today()
            elif date.lower() == 'yesterday':
                date = date.today() - timedelta(days=1)
            else:
                days_of_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
                if date.lower() in days_of_week:
                    days_ahead = (days_of_week.index(date.lower()) - date.today().weekday() + 7) % 7
                    date = date.today() + timedelta(days=days_ahead)

        try:
            loc = event_soup.find(class_='location-info__address').text.split('\n')
            venue = loc[0]
            address = loc[1]
        except AttributeError:
            venue = None
            address = None
        
        try:
            price = event_soup.find(class_='conversion-bar__panel-info').text
        except AttributeError:
            price = None
        
        if not end_date:
            end_date = date
        
        events.append({
            'Event Title': title,
            'Event Description': description,
            'Event Start Date': date,
            'Event End Date': end_date,
            'Event Start Time': start_time,
            'Event End Time': end_time,
            'Event Venue': venue,
            'Venue Address': address,
            'link': event_url
        })
    
    return events

all_events = []
link = "https://www.eventbrite.com/d/tx--dallas/free--business--events/?page="

# Visit Eventbrite website
for i in range(4, 61):
    path = link + str(i + 1)
    response = requests.get(path)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    time.sleep(5)  # Adjust sleep time as necessary to ensure page loads
    
    all_events.extend(extract_event_details(soup, path))
    print(f"Page {i + 1} completed")

# Save data to CSV
df = pd.DataFrame(all_events)
df.to_csv('Dallas_Events.csv', index=False)

print("Scraping complete. Data saved to Dallas_Events.csv")

No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=5
Page 5 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=6
Page 6 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=7
Page 7 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=8
Page 8 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=9
Page 9 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=10
Page 10 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=11
Page 11 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=12
Page 12 completed
No events found on https://www.eventbrite.com/d/tx--dallas/free--business--events/?page=13
Page 13 completed
No events found on https://ww

KeyboardInterrupt: 

In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import csv
from datetime import date, timedelta
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup


# Initialize WebDriver
driver_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome()

# Define function to extract event details

df = pd.DataFrame()
link = "https://www.eventbrite.com/d/tx--dallas/all-events/?page="

# Visit Eventbrite website
for i in range(0, 500):
    path = link + str(i + 1)
    driver.get(path)
    
    response = requests.get(path)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(5)  # Adjust sleep time as necessary to ensure page loads

    try:
        events_class = driver.find_element(By.CLASS_NAME, 'SearchResultPanelContentEventCardList-module__eventList___1YEh_')
    except NoSuchElementException:
        print("Events class not found.")
        
    end_date = None

    event_lists = []
    events = []

    try:
        event_hrefs = events_class.find_elements(By.TAG_NAME, 'a')
        for x in range(0, len(event_hrefs), 2):
            event_lists.append(event_hrefs[x].get_property('href'))
    except NoSuchElementException:
        print("Event hrefs not found.")
        

    for url in event_lists:
        driver.get(url)
        
        try:
            price = soup.find(class_ = 'conversion-bar__body')
            if price:
                price = price.text
                
            price = soup.find(class_ = 'ticket-card-compact-size__price')
            if price:
                price = price.text
        except NoSuchElementException:
            wait = WebDriverWait(driver, 20)
            p = driver.find_elements(By.CLASS_NAME,'TicketCard-module__pricing___38cNv')
            price = p[0].text + p[-1].text
            #price = None

        
        try:
            button = driver.find_element(By.CSS_SELECTOR, "button[data-testid='view-event-details-button']")
            button.click()
            print('Button clicked')
            WebDriverWait(driver, 10).until(EC.url_changes(url))
        except (NoSuchElementException, TimeoutException):
            #print("View event details button not found or URL did not change.")
            pass

        try:
            title = driver.find_element(By.TAG_NAME, 'h1').text
        except NoSuchElementException:
            title = None

        try:
            description = driver.find_element(By.CLASS_NAME, 'summary').text
        except NoSuchElementException:
            description = None

        try:
            date_info = driver.find_element(By.CLASS_NAME, 'date-info__full-datetime').text.split(',')
            if len(date_info) == 2:
                date_and_time_info = date_info[1]
                x = date_and_time_info.split('·')
                if len(x) == 2:
                    date, time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]

                    s = time_info.split('-')
                    if len(s) == 2:
                        start_time, end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                        end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                    else:
                        start_time = time_info
                        end_time = None
                    start_time = start_time + " " + time_zone
                    if end_time:
                        end_time = end_time + " " + time_zone

                else:
                    d = date_and_time_info.split()
                    date = ' '.join(d[:3])
                    start_time = ' '.join(d[3:5])
                    end_time = ' '.join(d[6:8])
                    time_zone = d[-1]
            elif len(date_info) > 2:
                start = date_info[1].split('-')[0].split()
                date = ' '.join(start[:3])
                start_time = start[-1]
                end = date_info[2].split()
                end_date = ' '.join(end[:3])
                end_time = end[-2]
                time_zone = end[-1]
                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone
            else:
                date_and_time_info = date_info[0]
                date, time_info = date_and_time_info.split('·')[0], date_and_time_info.split('·')[1]
                s = time_info.split('-')
                if len(s) == 2:
                    start_time, end_time_info = time_info.split('-')[0], time_info.split('-')[1]
                    end_time, time_zone = end_time_info.split()[0], end_time_info.split()[1]
                else:
                    start_time = time_info
                    end_time = None
                start_time = start_time + " " + time_zone
                if end_time:
                    end_time = end_time + " " + time_zone
        except NoSuchElementException:
            date = None
            #date = driver.find_element(By.CLASS_NAME, 'start-date').text.split(',')[1]
            end_date = None

        if date and date.isalpha():
            if date.lower() == 'today':
                date = date.today()
            elif date.lower() == 'yesterday':
                date = date.today() - timedelta(days=1)
            else:
                days_of_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
                if date.lower() in days_of_week:
                    days_ahead = (days_of_week.index(date.lower()) - date.today().weekday() + 7) % 7
                    date = date.today() + timedelta(days=days_ahead)

        try:
            wait = WebDriverWait(driver, 20)
            loc = soup.find(class_ =  'location-info__address')
            if loc:
                loc = loc.text.split('\n')
                venue = loc[0]
                address = loc[1]
        except NoSuchElementException:
            venue = None
            address = None


        if not end_date:
            end_date = date

        #events.append()
        df_2 = pd.DataFrame({
            'Event Title': [title],
            'Event Description': [description],
            'Event Start Date': [date],
            'Event Start Time': [start_time],
            'Event End Time': [end_time],
            'Event Cost': [price],
            'Event Venue': [venue],
            'Venue Address': [address],
            'link': [url]
        })
        df =  pd.concat([df,df_2],ignore_index=True)
    print(f"Page {i + 1} completed")
        #print(events)



all_events = []
#df = extract_event_details()
    

# Save data to CSV
# df = pd.DataFrame(all_events)
# df.to_csv('Dallas_Events.csv', index=False)

# # Close the driver
driver.quit()

print("Scraping complete. Data saved to Dallas_Events.csv")

Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed


KeyboardInterrupt: 

In [64]:
#df = pd.DataFrame(all_events)
df.to_csv('Dallas_Events_3_7.csv', index=False)

# # Close the driver

In [68]:
df.head(30)

,Event Title,Event Description,Event Start Date,Event End Date,Event Start Time,Event End Time,Event Cost,Event Venue,Venue Address,link
0,Dallas Juneteenth Festival,Commenerating the Juneteenth Holiday with live...,June 16,June 16,12 CDT,6pm CDT,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/dallas-juneteenth...
1,Juneteenth Soul Food Festival,Celebrating the Juneteenth Festival with Great...,June 16,June 16,12 CDT,6pm CDT,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/juneteenth-soul-f...
2,ELEVATED COLORS: RnB Rave (DTX),"Performances by Vanilla Trill, Crypto Castro, ...",June 16,June 16,5 CDT,10pm CDT,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/elevated-colors-r...
3,The Dallas Super Show - 2024,LMP Events returns to Dallas with The Dallas S...,June 9,June 16,12 CDT,6pm CDT,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/the-dallas-super-...
4,2024 Dallas Soul Flower Music Fest,The most beautiful event of the year,June 15,June 16,2 CDT,9pm CDT,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/2024-dallas-soul-...
5,BISCITS - Stereo Live Dallas,"BISCITS - Stereo Live Dallas Tickets, Parking ...",June 15,June 16,10pm 16,June 16,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/biscits-stereo-li...
6,SNAILS - Stereo Live Dallas,"SNAILS - Stereo Live Dallas Tickets, Parking P...",June 14,June 16,10pm 15,June 15,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/snails-stereo-liv...
7,MARKUS SCHULZ - Stereo Live Dallas,"MARKUS SCHULZ - Stereo Live Dallas Tickets, Pa...",June 21,June 16,10pm 22,June 22,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/markus-schulz-ste...
8,Cigars & Sundresses DAY Party @ Sandaga 813,Another DFW Favorite DAY Party,June 8,June 16,3 CDT,8pm CDT,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/cigars-sundresses...
9,CASH CASH - Stereo Live Dallas,"CASH CASH - Stereo Live Dallas Tickets, Parkin...",June 7,June 16,10pm 8,June 8,None,The Mason Dallas,"115 South Beckley Avenue Dallas, TX 75203 Unit...",https://www.eventbrite.com/e/cash-cash-stereo-...


In [63]:
len(df)

89